In [ ]:
import SALPY_ATPtg
import SALPY_ATMCS
import SALPY_ATPneumatics
import SALPY_ATHexapod
import SALPY_ATAOS

In [ ]:
from lsst.ts import salobj
import asyncio

In [ ]:
import numpy as np

In [ ]:
r1 = salobj.Remote(SALPY_ATPneumatics)

In [ ]:
r2 = salobj.Remote(SALPY_ATAOS)

In [ ]:
r3 = salobj.Remote(SALPY_ATHexapod)

In [ ]:
def get_callback_for(name):
    def print_pressure_callback(id_data):
        print(f'{name} {id_data.pressure}')
    return print_pressure_callback

def print_evt_hexapodCorrectionStarted_callback(id_data):
    print('CorrectionStarted:',id_data.azimuth, id_data.elevation, 
          id_data.hexapod_x, 
          id_data.hexapod_y, 
          id_data.hexapod_z, 
          id_data.hexapod_u, 
          id_data.hexapod_v, 
          id_data.hexapod_w)

def print__moveToPosition_callback(id_data):
    print('-->', 
          id_data.positionX, 
          id_data.positionY, 
          id_data.positionZ, 
          id_data.positionU, 
          id_data.positionV, 
          id_data.positionW)
    
def print_log_message(id_data):
    print(id_data.message)

def print_detailed_state(id_data):
    print(id_data.substate)
    
def print_error_code(id_data):
    print(id_data.errorCode,id_data.errorReport)

In [ ]:
r1.evt_m1SetPressure.callback = get_callback_for("ATPneu:M1")
r1.evt_m2SetPressure.callback = get_callback_for("ATPneu:M2")
r2.evt_m1CorrectionStarted.callback = get_callback_for("ATAOS:M1:")
r2.evt_m2CorrectionStarted.callback = get_callback_for("ATAOS:M2")
r3.evt_positionUpdate.callback = print__moveToPosition_callback
r2.evt_hexapodCorrectionStarted.callback = print_evt_hexapodCorrectionStarted_callback

In [ ]:
r2.evt_logMessage.callback = print_log_message

In [ ]:
r3.evt_logMessage.callback = print_log_message

In [ ]:
r3.evt_errorCode.callback = print_error_code

In [ ]:
r2.evt_detailedState.callback = print_detailed_state

In [ ]:
await asyncio.sleep(300.)

In [ ]:
await asyncio.sleep(300.)

In [ ]:
asyncio.get_event_loop().run_forever()

In [ ]:
asyncio.get_event_loop().run_until_complete(asyncio.sleep(10.))

In [ ]:
salobj.State(r1.evt_summaryState.get().summaryState)

In [ ]:
salobj.State(r2.evt_summaryState.get().summaryState)

In [ ]:
salobj.State(r3.evt_summaryState.get().summaryState)

In [ ]:
r3.cmd_start.set(settingsToApply='Default1')
await r3.cmd_start.start()

In [ ]:
await r2.cmd_enable.start()

In [ ]:
val = r3.evt_settingsAppliedTcp.get()

In [ ]:
print(val.ip, val.port)

In [ ]:
m1_correction_ataos = r2.evt_m1CorrectionStarted.get_oldest()
m2_correction_ataos = r2.evt_m2CorrectionStarted.get_oldest()

In [ ]:
print(m1_correction_ataos.pressure)
print(m2_correction_ataos.pressure)

In [ ]:
m1_setpressure_atpneumatics=r1.evt_m1SetPressure.get_oldest()
m2_setpressure_atpneumatics=r1.evt_m2SetPressure.get_oldest()

In [ ]:
print(m1_setpressure_atpneumatics.pressure)
print(m2_setpressure_atpneumatics.pressure)

In [ ]:
while True:
    coro1 = r1.evt_m1SetPressure.next(flush=True)
    coro2 = r1.evt_m2SetPressure.next(flush=True)
    ret = await asyncio.gather(coro1, coro2)
    print(ret[0].pressure, ret[1].pressure)

In [ ]:
c = r2.evt_correctionEnabled.get()

In [ ]:
c.focus, c.hexapod, c.m1, c.m2

In [ ]:
r2.cmd_disableCorrection.set(disableAll=True)

In [ ]:
await r2.cmd_disableCorrection.start()

In [ ]:
r2.cmd_enableCorrection.set(m2=True, m1=False, hexapod=False, focus=False)
# r2.cmd_enableCorrection.set(m1=True)

In [ ]:
await r2.cmd_enableCorrection.start()

In [ ]:
asyncio.get_event_loop().run_until_complete(asyncio.sleep(10.))

In [ ]:
c = r2.evt_correctionEnabled.get()

In [ ]:
c.focus, c.hexapod, c.m1, c.m2

In [ ]:
r2.evt_m2CorrectionStarted.callback

In [ ]:
while True:
    log = await r2.evt_logMessage.next(flush=False, timeout=1.)
    print(log.message)

In [ ]:
d = r2.evt_detailedState.get()

In [ ]:
d.substate

In [ ]:
d.substate & (1 << 2) != 0

In [ ]:
1<<4

In [ ]:
await r2.cmd_enable.start()

In [ ]:
await r3.cmd_disable.start()

In [ ]:
await r3.cmd_standby.start()

In [ ]:
await r3.cmd_exitControl.start()

In [ ]:
await r3.cmd_start.start()

In [ ]:
await r3.cmd_enable.start()

In [ ]:
await r3.evt_heartbeat.next(flush=True)

In [ ]:
asyncio.get_event_loop().run_until_complete(asyncio.sleep(5.))

In [ ]:
r3.cmd_moveToPosition.set(x=np.random.random(), y=np.random.random(), z=np.random.random())

In [ ]:
a = 6.41761786652
for i in range(5):
    r3.cmd_moveToPosition.set(x=0., y=0., z=a+1e-3)
    await r3.cmd_moveToPosition.start(timeout=5)

In [ ]:
await asyncio.sleep(300.)

In [ ]:
await asyncio.sleep(3.)

In [ ]:
salobj.State(r3.evt_summaryState.get().summaryState)

In [ ]:
await r3.cmd_disable.start()

In [ ]:
await r3.cmd_standby.start()

In [ ]:
r3.cmd_start.set(settingsToApply="Default1")

In [ ]:
await r3.cmd_start.start()

In [ ]:
await r3.cmd_enable.start(timeout=5)

In [ ]:
await r3.cmd_setLogLevel.start(timeout=10)

In [ ]:
r3.cmd_setLogLevel.set(level=10)

In [ ]:
r2.cmd_start.set(settingsToApply="test")

In [ ]:
await r2.cmd_start.start()

In [ ]:
await r2.cmd_disablk.start(timeout=5)